In [1]:
# based on https://github.com/chkwon/TrafficAssignment.jl

include(joinpath("/home/xiangyu/research/InverseVIsTraffic/Julia_files/load_network_uni_class.jl"))

using Graphs

function create_graph(start_node, end_node)
    @assert Base.length(start_node)==Base.length(end_node)

    no_node = max(maximum(start_node), maximum(end_node))
    no_arc = Base.length(start_node)

    graph = simple_inclist(no_node)
    for i=1:no_arc
        add_edge!(graph, start_node[i], end_node[i])
    end
    return graph
end

function get_vector(state, origin, destination, link_dic)
    current = destination
    parent = -1
    x = zeros(Int, maximum(link_dic))

    while parent != origin
        parent = state.parents[current]

        link_idx = link_dic[parent,current]

        if link_idx != 0
            x[link_idx] = 1
        end

        current = parent
    end

    return x
end

ta_data = load_ta_network("East_Massachusetts_Apr_AM")

# unpacking data from ta_data
network_name = ta_data.network_name

number_of_zones = ta_data.number_of_zones
number_of_nodes = ta_data.number_of_nodes
first_thru_node = ta_data.first_thru_node
number_of_links = ta_data.number_of_links

start_node = ta_data.start_node
end_node = ta_data.end_node
capacity = ta_data.capacity

free_flow_time = ta_data.free_flow_time
B = ta_data.B
power = ta_data.power
speed_limit = ta_data.speed_limit
toll = ta_data.toll
link_type = ta_data.link_type
number_of_zones = ta_data.number_of_zones
total_od_flow = ta_data.total_od_flow
travel_demand = ta_data.travel_demand
od_pairs = ta_data.od_pairs

toll_factor = ta_data.toll_factor
distance_factor = ta_data.distance_factor

best_objective = ta_data.best_objective

# preparing a graph
graph = create_graph(start_node, end_node)
link_dic = sparse(start_node, end_node, 1:number_of_links);

In [2]:
free_flow_time

24-element Array{Float64,1}:
 0.294839
 0.314894
 0.437603
 0.44623 
 0.136564
 0.162954
 0.512348
 0.528436
 0.226178
 0.236022
 0.187778
 0.193503
 0.232767
 0.238257
 0.166785
 0.164794
 0.330009
 0.329602
 0.178723
 0.177963
 0.245841
 0.244939
 0.18718 
 0.192552

In [3]:
start_node

24-element Array{Int64,1}:
 1
 2
 1
 3
 2
 3
 2
 4
 3
 5
 3
 6
 4
 5
 5
 6
 4
 8
 5
 7
 6
 7
 7
 8

In [4]:
end_node

24-element Array{Int64,1}:
 2
 1
 3
 1
 3
 2
 4
 2
 5
 3
 6
 3
 5
 4
 6
 5
 8
 4
 7
 5
 7
 6
 8
 7

In [5]:
capacity

24-element Array{Float64,1}:
 5881.03
 5723.73
 7257.79
 7352.89
 7259.77
 7367.74
 6144.81
 6141.05
 7977.67
 7839.68
 7574.68
 6560.51
 5758.19
 6409.61
 5930.99
 4755.89
 6000.0 
 6000.0 
 7240.18
 7284.73
 6927.24
 6690.82
 6000.0 
 6000.0 

In [6]:
function MSA(coeffs) 
    polyEval(coeffs, pt) = sum([coeffs[i] * pt^(i-1) for i = 1:length(coeffs)]) 

    function BPR(x)
        bpr = similar(x)
        for i=1:length(bpr)
            bpr[i] = free_flow_time[i] * polyEval( coeffs, (x[i]/capacity[i]) ) 
        end
        return bpr
    end

    function all_or_nothing(travel_time)
        state = []
        path = []
        x = zeros(size(start_node))

        for r=1:size(travel_demand)[1]
            # for each origin node r, find shortest paths to all destination nodes
            state = dijkstra_shortest_paths(graph, travel_time, r)

            for s=1:size(travel_demand)[2]
                # for each destination node s, find the shortest-path vector
                # load travel demand
                x = x + travel_demand[r,s] * get_vector(state, r, s, link_dic)
            end
        end

        return x
    end

    # Finding a starting feasible solution
    travel_time = BPR(zeros(number_of_links))
    xl = all_or_nothing(travel_time)

    max_iter_no = 1e3
    l = 1
    #average_excess_cost = 1
    tol = 1e-5

    while l < max_iter_no
        l += 1

        xl_old = xl

        # Finding yl
        travel_time = BPR(xl)

        yl = all_or_nothing(travel_time)

        xl = xl + (yl - xl)/l

        xl_new = xl

        relative_gap = norm(xl_new - xl_old, 1) / norm(xl_new, 1)

        if relative_gap < tol
            break
        end
    end
    
    return xl
end

MSA (generic function with 1 method)

In [7]:
# getting the coefficients of the costs
coeffs_dict_Apr_AM = readstring("../temp_files/coeffs_dict_Apr_AM.json")
coeffs_dict_Apr_AM = JSON.parse(coeffs_dict_Apr_AM)

xl = Dict{}()

# deg_grid = [4:8]
# c_grid = .5:.5:3.
# lamb_grid = [10. .^(-3:4)]

deg_grid = 8
c_grid = .5
lamb_grid = 10. .^(4)

for deg in deg_grid
    for c in c_grid
        for lam in lamb_grid
            coeffs_1 = coeffs_dict_Apr_AM["($(deg),$(c),$(lam),1)"]
            coeffs_2 = coeffs_dict_Apr_AM["($(deg),$(c),$(lam),2)"]
            coeffs_3 = coeffs_dict_Apr_AM["($(deg),$(c),$(lam),3)"]
            xl[(deg, c, lam, 1)] = MSA(coeffs_1)
            xl[(deg, c, lam, 2)] = MSA(coeffs_2)
            xl[(deg, c, lam, 3)] = MSA(coeffs_3)
        end
    end
end

In [8]:
lamb_grid

10000.0

In [9]:
outfile = open("../temp_files/uni-class_traffic_assignment_MSA_flows_Apr_AM.json", "w")

JSON.print(outfile, xl)

close(outfile)

In [10]:
xl

Dict{Any,Any} with 3 entries:
  (8,0.5,10000.0,3) => [3036.13,2668.1,1111.59,5901.75,4503.81,6786.31,3190.99,…
  (8,0.5,10000.0,1) => [3048.57,2668.1,1099.14,5901.75,4516.25,6786.31,3190.99,…
  (8,0.5,10000.0,2) => [3048.57,2668.1,1099.14,5901.75,4516.25,6786.31,3190.99,…

In [11]:
xl[(8,0.5,10000.0,2)]

24-element Array{Float64,1}:
  3048.57
  2668.1 
  1099.14
  5901.75
  4516.25
  6786.31
  3190.99
  3762.82
  6898.07
 10142.3 
  6811.61
  8182.62
  5772.62
  1992.57
  6540.28
  2437.24
  5181.99
  1684.09
 10518.5 
  4438.87
  9178.22
  5699.82
  3814.7 
  3672.63